##Installation OpenFace

In [ ]:
import os
from os.path import exists, join, basename, splitext

################# Need to revert back to CUDA 10.0 ##################
# Thanks to http://aconcaguasci.blogspot.com/2019/12/setting-up-cuda-100-for-mxnet-on-google.html
#Uninstall the current CUDA version
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

#Download CUDA 10.0
!wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
#install CUDA kit dpkg
!dpkg -i -y cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-0
#Slove libcurand.so.10 error
!wget --no-clobber http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
#-nc, --no-clobber: skip downloads that would download to existing files.
!apt install ./nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!apt-get update
####################################################################

git_repo_url = 'https://github.com/TadasBaltrusaitis/OpenFace.git'
project_name = splitext(basename(git_repo_url))[0]
# clone openface
!git clone -q --depth 1 $git_repo_url

# install new CMake becaue of CUDA10
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

# Get newest GCC
!sudo apt-get update
!sudo apt-get install build-essential 
!sudo apt-get install g++-8

# install python dependencies
!pip install -q youtube-dl

# Finally, actually install OpenFace
!cd OpenFace && bash ./download_models.sh && sudo bash ./install.sh

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/core/mat_operations/mat_operations.cpp  
   creating: opencv-4.1.0/samples/cpp/tutorial_code/core/mat_the_basic_image_container/
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/core/mat_the_basic_image_container/mat_the_basic_image_container.cpp  
   creating: opencv-4.1.0/samples/cpp/tutorial_code/features2D/
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/features2D/AKAZE_match.cpp  
   creating: opencv-4.1.0/samples/cpp/tutorial_code/features2D/AKAZE_tracking/
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/features2D/AKAZE_tracking/planar_tracking.cpp  
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/features2D/AKAZE_tracking/stats.h  
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/features2D/AKAZE_tracking/utils.h  
   creating: opencv-4.1.0/samples/cpp/tutorial_code/features2D/Homography/
  inflating: opencv-4.1.0/samples/cpp/tutor

##Décision Dict



In [ ]:
%%capture
!pip install treelib

In [ ]:
import json
from treelib import Node, Tree

file = "/POC/decision_dict.json"

# List of used Action Unis
AUs = ['1', '2', '4', '5', '6', '7','9', '10', '12', '14', '15', '17', '18', '20', '25', '43']

#Class handling the Decision dictionary
class DecisionDict:

    # Constructor
    # file: path to the file used to create the decision dict
    # mode: mode used for decision (see get decision)
    def __init__(self, file, mode="base"):
        with open(file, 'r') as f:
            self.dict = json.load(f)
        self.set_mode(mode)

    # Set mode
    def set_mode(self,mode):
        self.mode = mode

    # Format the action units output to be used by the decision dict, returning a string
    # output: output of the action units detectors (binary list)
    def format_output(self, output):
        output.sort()
        formatted_output = ""
        for i, elt in enumerate(output):
            if elt == 1:
                formatted_output += AUs[i] + " "
            #formatted_output += str(elt)
            #if i < len(output) - 1:
            #    formatted_output += " "
        return formatted_output[0:-1]

    # Format the decision returning a dict with key is emotion and value is the percentage
    # decision: list of dictionaries
    def format_decision(self, decision):
        res = {}
        print(decision)
        for elt in decision:
            res[elt["emotion"]] = float(elt["percentage"])
        return res

    # Take the action unit detectors output and give the emotions as a dict
    # Aus: binary list containing the results of action units detectors
    def get_decision(self,AUs):
        formatted_output = self.format_output(AUs)
        print(formatted_output,self.mode)
        if formatted_output in list(self.dict.keys()):
            # base: return all the emotions
            if self.mode == "base":
                return(self.format_decision(self.dict[formatted_output]))
            # max: return only the emotion with highest probability
            if self.mode == "max":
                return(self.format_decision(self.dict[formatted_output][0]))
        else:
            return({})

##Décision Tree



In [ ]:
import json
from treelib import Node, Tree
import math
file = "/POC/decision_dict.json"

# List of used Action Unis
AUs = ['1', '2', '4', '5', '6', '7','9', '10', '12', '14', '15', '17', '18', '20', '25', '43']

#Class handling the Decision tree
class DecisionTree:

    # Constructor
    # file: path to the file used to create the decision dict
    # mode: mode used for decision (see get decision)
    def __init__(self, file, mode="minimal_distance"):
        self.tree = self.init_tree(file)
        self.mode = mode

    # Convert Action units detectors' outputs into tree's node identifier
    # AUs: action units detector output (binary list)
    def AUs2Nodes(self,AUs):
        res = []
        tmp = ""
        for i,AU in enumerate(AUs):
            if i > 0:
                tmp += ";"
            tmp += AU
            res.append(tmp)
        return res

    # Create a node tag from emotion dict
    # emotions: list of dictionary with emotion and percentage as keys and emotion name and associated percentage as values
    # return node tag as a string (format: 'emotion1:percentage1;...;emotionN:percentageN'
    def emotion2Nodes(self,emotions):
        res = ""
        for i,emotion in enumerate(emotions):
            res += emotion["emotion"]+":"+emotion["percentage"]
            if i < len(emotions) - 1:
                res += ";"
        return(res)


    # Give the closest leave
    # path_to_leaves:
    def get_minimal_leaves(self, path_to_leaves):
        print('patl',path_to_leaves)
        path_to_leaves.sort(key=lambda x: len(x), reverse = False)
        print(path_to_leaves)
        return(path_to_leaves[0][-1])

    # Give emotion result from a tree node
    # node: a tree node
    # return dictionary with emotion as key and percentage as value
    def node2result(self,node):
        node_data = self.tree.get_node(node).tag
        print(node_data)
        return {  y.split(':')[0]:float(y.split(':')[1]) for y in node_data.split(";") }

    # Give the mean of emotions' percentage
    # results: dictionary with emotion as key and percentage as value
    def meanresult(self,results):
        mean_result = { }
        tmp = {}
        print(results)
        for key, value in results.items():
            if key not in list(mean_result.keys()):
                mean_result[key] = (value)
            if key not in list(tmp.keys()):
                tmp[key] = 1
            mean_result[key] += (value)
            tmp[key] += 1
        for key in list(mean_result.keys()):
            mean_result[key] /= tmp[key]
        return(mean_result)

    #Create a tree giving the json file listing action units combinations and their associated emotions
    # set the tree attribute
    def init_tree(self,file):
        count = 50 # Used for leaves ID to avoid duplicate ID with action unit combinations leading to similare results
        # Start arbitrary at 50 to avoid duplicate ID with node of action units like '43'
        tree = Tree()
        tree.create_node("AU","AU")
        with open(file,'r') as f:
            dict = json.load(f)
            for key, value in dict.items():
                nodes = self.AUs2Nodes(key.split())
                if nodes[0] not in [ x.tag for x in tree.all_nodes() ]:
                    tree.create_node(nodes[0],nodes[0],parent="AU")#nodes[0], nodes[0], parent="AU")
                for i in range(1,len(nodes)):
                    if nodes[i] not in [ x.tag for x in tree.all_nodes() ]:
                        tree.create_node(nodes[i],nodes[i],parent=nodes[i-1])
                emotion_leaf = self.emotion2Nodes(value)
                tree.create_node(emotion_leaf,count,nodes[len(nodes)-1])
                count += 1
        return tree

    # Remove the nth AU from node_id (ex trunc_node_id('1;2;43',2) return '1;43')
    # node_id: node_id as string, the node_id format is 'au1;au2;..;auN'
    # n: int, the elt to be removed from node_id
    # return string
    def trunc_node_id(self,node_id,n):
        res = ""
        for i, id in enumerate(node_id.split(";")):
            if not i == n:
                res += id + ";"
        return res[0:-1]

    # Take the action unit detectors output and give the emotions as a dict
    # Aus: binary list containing the results of action units detectors
    def get_decision(self,AU_output):
        node_id = ""
        for i,elt in enumerate(AU_output):
            if elt == 1:
                node_id += str(AUs[i]) + ";"
        node_id = node_id[0:-1]
        print(node_id)
        final_node_id = None
        if node_id not in [ x.tag for x in self.tree.all_nodes() ]:
            for i in range(0,len(node_id.split(";"))):
                trunc_node_id = self.trunc_node_id(node_id,i)
                if trunc_node_id in [ x.tag for x in self.tree.all_nodes() ]:
                    final_node_id = trunc_node_id
                    break
        else:
            final_node_id = node_id
        if final_node_id:
            subtree = Tree(self.tree.subtree(final_node_id))
            leaves = subtree.leaves()
            # minimal_distance: give the result of the closest leave
            if self.mode == "minimal_distance":
                paths_to_leaves = subtree.paths_to_leaves()
                decision = self.meanresult(self.node2result(self.get_minimal_leaves(paths_to_leaves)))
            # mean the results of all the leaves reachable from the node related to the node_id we get from action units detected
            if self.mode == "mean_results":
                decision = leaves
            return(decision)
        else:
            return {}


##QUEUE

In [ ]:

class Queue:

    def __init__(self,size):
        self.size = size
        self.data = []
        self.weights = 0
        for i in range(1, size+1):
            self.weights += i

    def add(self,value):
        self.data.append(value)
        if len(self.data) > self.size:
            self.data.pop(0)

    def get(self):
        res = 0
        for i in range(0, len(self.data)):
            res += (i+1) * self.data[i]
        res /= self.weights
        return res


##Utils

In [ ]:
%%capture
!pip install unidecode

In [ ]:
import json
import unidecode
import math


#Given the csv file associating action units combination and emotions it creates the corresponding dictionary
def create_decision_dict(file):
    decision_dict = {}
    with open(file,'r') as f:
        for line in f.readlines():
            line = line.strip("\n")
            data = line.split(";")
            combi = data[0]
            decision_dict[combi] = []
            for i in range(1,len(data),2):
                if data[i]:
                    emotion = unidecode.unidecode(data[i])
                    percentage = data[i+1]
                    decision_dict[combi].append({"emotion": emotion, "percentage": percentage})
        with open("decision_dict.json", 'w') as out:
            json.dump(decision_dict,out)

# Get euclidean distance between two points
def get_dist(posA,posB):
    return(math.sqrt(math.pow(posA[0]-posB[0],2) + math.pow(posB[1] - posA[1],2)))

#create_decision_dict("/data/Datasets/AUs/Significations_combis_DT pourcentages_2022.csv")

##Emotion Detector

In [ ]:
%%capture
!pip install onnx
!pip install onnxruntime
!pip install argparse

In [ ]:
import sys
sys.path.append("..")
import cv2 as cv
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
import onnx
import onnxruntime as ort
import argparse
import os

#Class responsible of emotion detection
# Constructor
# AU_detector_weights: path to the folder containing the Action Units detection weights
# decision_file: path to the json containing the decision dict
# face_detection_cfg: path to the cfg file of the face_detection model
# face_detection_weights: path to the weights file of the face_detection model
# decision_tool: decision tool to use: dict or tree
# decision_mode: see the decision classes to check the available classes
class Emotion_detector:
    def __init__(self, decision_file,face_detection_cfg="",face_detection_weights="",decision_tool="dict", decision_mode="base"):
        self.decision_model = None
        self.face_detection_model = None
        self.emotion_dynamic = 5 # 'memory' length of the emotions
        self.set_decision(decision_tool, decision_file, decision_mode) # Set the decision mode: dict or tree
        self.decision_tool = decision_tool # Set the decision object: decision_dict or decision_tree (see decision classes)
        self.init_face_detector(face_detection_cfg, face_detection_weights, 512) # Init the TinyYolo face detection model
        self.resize = transforms.Resize(224) # Transformation used for action unit detection model
        self.transforms = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.599604128975922, 0.45806012311249106, 0.40484804820393744), (0.2413158196573673, 0.21111772733232814, 0.20150280760684183))
        ]) # Not used for now
        self.faces_centroid = [] # list of faces centroid
        self.emotion = [] # list of emotion, each index is associated with one face

    # Init the Yolo face detection model
    # face_detection_cfg: path to the cfg file of the model
    # face_detection_weights: path to the model's weights' file
    # input_size: int used for image resizing
    def init_face_detector(self,face_detection_cfg,face_detection_weights, input_size):
        print(face_detection_cfg, face_detection_weights)
        self.face_detection_model = cv.dnn_DetectionModel(face_detection_cfg, face_detection_weights)
        self.face_detection_model.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
        self.face_detection_model.setPreferableBackend(cv.dnn.DNN_TARGET_CPU)
        self.face_detection_model.setInputSize(input_size, input_size)
        self.face_detection_model.setInputScale(1.0 / 255)
        self.face_detection_model.setInputSwapRB(True)
        print(self.face_detection_model)

    # Init the action units detection model:
    # version: not used for now, needed to handle multiple model version
    # weight_path: path to the model's weights' file

    # Set the decision mode and init the decision tool
    # decision_tool: dict/tree which mode to use
    # decision_file: path to the json file representing the decision process
    # decision_mode: string: see the decision_tree and decision_dict to see the available modes
    def set_decision(self, decision_tool, decision_file, decision_mode):
        if not hasattr(self,'decision_tool'):
            self.decision_tool = decision_tool
        elif decision_tool and decision_tool == self.decision_tool:
            print("Already using :" + decision_tool)
            pass
        if self.decision_tool == "dict":
            self.decision_model = DecisionDict(decision_file,mode=decision_mode)
        elif self.decision_tool == "tree":
            self.decision_model = DecisionTree(decision_file, mode=decision_mode)
        else:
            print("Error")

    # Take faces_position as input (list of tuple) and return the new faces
    # it tries to associate each face_position with the already existing face by
    # measuring the distance between their centroid
    # return the non matched faces as a list of tuple
    def find_new_faces(self,faces_position):
        new_faces = [ x for x in range(0,len(faces_position))]
        for posB in self.faces_centroid:
            min_dist = float('inf')
            min_index = 0
            for i, posA in faces_position:
                dist = get_dist(posA,posB)
                if dist < min_dist:
                    min_dist = dist
                    min_index = i
            new_faces.remove(min_index)
        return new_faces

    # Find the index of a face
    # face_position: tuple of int
    # return index as int, used to associate emotion with a face
    def find_centroid_index(self,face_position):
        min_dist = float('inf')
        min_index = 0
        for i, posB in enumerate(self.faces_centroid):
            dist = get_dist(face_position,posB)
            if dist < min_dist:
                min_dist = dist
                min_index = i
        return min_index

    # Updates the faces centroid, adding new face centroid if necessary
    # faces: list of faces (can really necessary)
    # faces_position: list of tuple
    def update_faces_centroid(self,faces,faces_position):
        new_faces_index = []
        if len(self.faces_centroid) < len(faces):
            new_faces_index = self.find_new_faces(faces_position)
        for i,pos in enumerate(faces_position):
            if not i in new_faces_index:
                centroid_index = self.find_centroid_index(pos)
                self.faces_centroid[centroid_index] = ((self.faces_centroid[centroid_index][0] + pos[0])/2.0, (self.faces_centroid[centroid_index][1] + pos[1])/2.0)
            else:
                self.faces_centroid.append((pos[0],pos[1]))

    # Update the emotions
    # emotion: dictionary of emotion
    # face_position: not used yet
    # emotion_index: index of the face to which the emotion are associated
    # set the emotion attribute
    def update_emotions(self,emotion, face_position, emotion_index):
        print("Emotions",emotion,emotion_index)
        if len(self.emotion)< emotion_index +1:
            self.emotion.append({})
        for key,value in emotion.items():
            if key not in list(self.emotion[emotion_index].keys()):
                self.emotion[emotion_index][key] = Queue(self.emotion_dynamic)
                self.emotion[emotion_index][key].add(value)
            else:
                self.emotion[emotion_index][key].add(value) #= (self.emotion[emotion_index][key] + value)
        print("emo:",self.emotion)
        for key, value in self.emotion[emotion_index].items():
            if key not in list(emotion.keys()):
                self.emotion[emotion_index][key].add(0.0)

    # get emotion giving an index and a mode
    # index: index of the face to which the emotion is associated
    # mode: only max for now: give the emotion with max percentage
    # Need a function to average the emotions
    def get_emotion(self, index, mode="max"):
        if mode == "max":
            max_val = 0
            max_emo = ""
            for key,value in self.emotion[index].items():
                val = value.get()
                if val > max_val:
                    max_val = val
                    max_emo = key
            return (max_emo,max_val)

    # Giving a frame return the detected faces and their respective positions in the frame, using TinyYolo
    # img: cvMat of the frame
    def get_faces(self,img):
        print("get_faces")
        classes, confidences, boxes = self.face_detection_model.detect(img, confThreshold=0.1, nmsThreshold=0.4)
        faces = []
        faces_position = []
        if (not len(classes) == 0):
            for classId, confidence, box in zip(classes, confidences, boxes):
                if classId in [0]:
                    left, top, width, height = box
                    faces.append(img[top:top + height, left:left + width])
                    faces_position.append((left, top))
        return (faces, faces_position)

    # Process a face before giving it to the action unit detection model
    # face: cvMat
    # return transformed image as a numpy array
    def process_face(self,face):
        face_rgb = cv.cvtColor(face, cv.COLOR_BGR2RGB)

        img = Image.fromarray(face_rgb)  # os.path.join(self.imgdir, fn))
        img.save('image.png')

        img = img.convert('RGB')
        img = self.resize(img)
        img = np.array(img).astype(np.float32)
        img_tensor = np.expand_dims(img,axis=0)
        img = img_tensor / 255. #[None,...]
        #img = self.transforms(img)


        return img



    # Giving a frame, do all the process of faces detections, action unit detection and action unit to emotion decision
    # return the emotions and their associated faces position
    def process_frame(self,frame):
        faces,faces_position = self.get_faces(frame)
        print("get_faces", len(faces))
        results = [("",0)]*len(faces)
        self.update_faces_centroid(faces,faces_position)
        for i, face in enumerate(faces):
            AUs = self.ExtractAu_OpenFace(self.process_face(face))#[None, ...].float()))
            print("AUs",AUs)
            emotion = self.decision_model.get_decision(AUs)
            emotion_index = self.find_centroid_index(faces_position[i])
            print(emotion,emotion_index)
            if emotion:
                self.update_emotions(emotion,faces_position[i], emotion_index)
                results[i] = self.get_emotion(emotion_index)
        return results, faces_position

    def extract_features_OpenFace(self,image, path_installed_openFace="/content/OpenFace", out_path='/content/Video/OpenFaceGeneration', static=" -au_static"):
        """
            Generate AUs using the OpenFace library

            :param video_name:[str] Name of the video to process
            :param: in_root_videos [str]: Path to the folder that contains all the videos to process
            :param: path_installed_openFace [str]: Path to the folder whwere we installed OpenFace
            :param out_path[str]: Path to save the AUs and extra material generated by OpenFace
            :param static[str]: au_static flag tells OpenFace not to perform dynamic calibration and to use only static models for AU prediction (see: https://github.com/TadasBaltrusaitis/OpenFace/wiki/Action-Units)

        """
        image_path="/content/image.png"


        command = os.path.join(path_installed_openFace, "build", "bin", "FaceLandmarkImg")+ static + " -f "+image_path+" -out_dir "+out_path
        os.system(command)


    def save_embs_complete(self,path_folder_OpenFace, out_path_embs):
        """
        Save Dataframes with only the AUs columns
            :param path_folder_OpenFace:[str] Path where the AUs wehre saved during the extraction process of OpenFace
            :param: out_path_embs [str]: Path to the save the new Dataframes generated that only contains onformation ofthe AUs.
        """
        res=[0]*16
        cols2select = ["AU01_c", "AU02_c", "AU04_c" ,"AU05_c", "AU06_c", "AU07_c", "AU09_c", "AU10_c", "AU12_c", "AU14_c", "AU15_c", "AU17_c", "AU20_c", "AU25_c", "AU45_c"]
        path_AU = os.path.join(path_folder_OpenFace, "image.csv")
        df_aus = pd.read_csv(path_AU, ",")
        df_aux = df_aus[cols2select]
        df_aux.to_csv(os.path.join(out_path_embs, "image.csv"), sep=";", header=True, index=False)
        i=0
        for column in df_aux.columns:
            res[i]=df_aux[column][0]
            i+=1
        print(res)
        return res




    # Giving a frame, do all the process of faces detections, action unit detection and action unit to emotion decision
    # return the emotions and their associated faces position
    def ExtractAu_OpenFace(self,ImagePath):
        parser = argparse.ArgumentParser(description="Configuration of setup and training process")
        parser.add_argument('-videos', '--videos_dir', type=str,  default=ImagePath,
                            help='Path with the embeddings to train/test the models')
        parser.add_argument('-out', '--out_dir', type=str,
                            help='Path to save the AUs & additional material generated by the OpenFace library',
                            default='/content/Video/OpenFaceGeneration')
        parser.add_argument('-outProcessed', '--out_dir_processed', type=str,
                            help='Path to save the AUs extracted from OpenFace after processing them',
                            default='/content/Video/OpenFaceAU')
        parser.add_argument('-openFace', '--openFace_path', type=str,  default="/OpenFace",
                            help='Path where you have installed/downloaded the OpenFace library')
        args = parser.parse_args(args=[])

        os.makedirs(args.out_dir_processed, exist_ok=True)
        os.makedirs(args.out_dir, exist_ok=True)

        self.extract_features_OpenFace(ImagePath)
        return self.save_embs_complete(args.out_dir, args.out_dir_processed)





In [ ]:
import sys
import cv2 as cv
import numpy as np
decision_tree = None

# Display emotion above faces
# img: cvMat on which emotion will be added
# emotions: list of emotions
# positions: list of faces coordinates
def display_info(img, emotions,positions):
    for i,pos in enumerate(positions):
        x,y = positions[i]
        print(x,y, emotions)
        cv.putText(img, emotions[i][0] + ":" + str(emotions[i][1]),(x,y-10), cv.FONT_HERSHEY_PLAIN,1, (255,255,255),2,cv.LINE_AA)
    return img

# main loop of the POC
# video_file: path to the the video to process
# mode: display/writing : writing add the possibility to write the processed video
# output_file: path where to write the video in "writing" mode
# fps: fps for the VideoWriter in "writing" mode
# frame_size: size of the video to write in "writing" mode
def main_loop(video_file, mode="display", output_file=None, fps=None, frame_size=None):
    emotion_detector = Emotion_detector( "decision_dict.json",
                                        face_detection_cfg="Yolo_weights/yolo.cfg",
                                        face_detection_weights="Yolo_weights/face-yolo.weights",decision_tool="tree", decision_mode="minimal_distance")
    cap = cv.VideoCapture(video_file)
    if mode == "writing":
        out = cv.VideoWriter(output_file, cv.VideoWriter_fourcc('M','J','P','G'),fps, frame_size)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            emotions, faces_position = emotion_detector.process_frame(frame)
            new_frame = display_info(frame, emotions, faces_position)
            #cv.imshow("results", new_frame)
            #cv.waitKey(1)
            if mode == "writing":
                out.write(new_frame)
        else:
            break
    cap.release()
    if mode == "writing":
        out.release()

main_loop('part1.mp4', mode="writing", output_file="test2.avi", fps=50, frame_size=(640,360))

NameError: ignored